In [73]:
import pandas as pd
import numpy as np
import os

### 1. Data Loading and Consolidation

This section handles the initial loading and combining of the dataset, which is split into multiple CSV files by year.

In [74]:
# Dynamically create a list of full file paths for all CSVs in the data directory.
# This scalable method automatically includes all data files without needing to hardcode their names.
all_csv_filepaths = [f'/content/datasets/csv/csv/{file}' for file in os.listdir('/content/datasets/csv/csv/')]

# Display the generated file paths to confirm they were located correctly.
all_csv_filepaths

['/content/datasets/csv/csv/2007.csv',
 '/content/datasets/csv/csv/2019.csv',
 '/content/datasets/csv/csv/2011.csv',
 '/content/datasets/csv/csv/2006.csv',
 '/content/datasets/csv/csv/2025.csv',
 '/content/datasets/csv/csv/2003.csv',
 '/content/datasets/csv/csv/2010.csv',
 '/content/datasets/csv/csv/2024.csv',
 '/content/datasets/csv/csv/2005.csv',
 '/content/datasets/csv/csv/2012.csv',
 '/content/datasets/csv/csv/2014.csv',
 '/content/datasets/csv/csv/2004.csv',
 '/content/datasets/csv/csv/2015.csv',
 '/content/datasets/csv/csv/2013.csv',
 '/content/datasets/csv/csv/2009.csv',
 '/content/datasets/csv/csv/2021.csv',
 '/content/datasets/csv/csv/2008.csv',
 '/content/datasets/csv/csv/2001.csv',
 '/content/datasets/csv/csv/2020.csv',
 '/content/datasets/csv/csv/2018.csv',
 '/content/datasets/csv/csv/2017.csv',
 '/content/datasets/csv/csv/2016.csv',
 '/content/datasets/csv/csv/2022.csv',
 '/content/datasets/csv/csv/2023.csv',
 '/content/datasets/csv/csv/2002.csv']

In [75]:
# Read and concatenate all the located CSV files into a single, master DataFrame.
# Using `map(pd.read_csv, ...)` is a memory-efficient method for this operation.
# `ignore_index=True` ensures the final DataFrame has a clean, continuous index from 0 to n-1.
commodity_prices_df = pd.concat(map(pd.read_csv, all_csv_filepaths), ignore_index=True)

### 2. Data Type Inspection and Correction

Before any cleaning or analysis can be performed, it is crucial to ensure that each column has the correct data type. This section focuses on identifying and correcting these types.

#### Initial Data Inspection

In [76]:
# Display the first few rows of the DataFrame to get a qualitative sense of the data.
commodity_prices_df.head()

,State,District,Market,Commodity,Variety,Grade,Arrival_Date,Min_Price,Max_Price,Modal_Price,Commodity_Code
0,Andhra Pradesh,Anantapur,Tenakallu,Ground Nut Seed,Ground Nut Seed,Medium,2007-01-01,3000.0,4000.0,3500.0,268
1,Andhra Pradesh,Anantapur,Tenakallu,Groundnut,Local,Medium,2007-01-01,2300.0,2500.0,2400.0,10
2,Andhra Pradesh,Krishna,Kanchekacherla,Green Gram (Moong)(Whole),Local,FAQ,2007-01-01,2800.0,2850.0,2825.0,9
3,Andhra Pradesh,Krishna,Kanchekacherla,Maize,Hybrid,FAQ,2007-01-01,650.0,660.0,655.0,4
4,Andhra Pradesh,Krishna,Kanchekacherla,Paddy (Dhan)(Common),Samba Masuri,FAQ,2007-01-01,900.0,950.0,925.0,2


In [77]:
# Display a concise summary of the DataFrame, including the data type of each column and memory usage.
# This initial inspection reveals that 'Arrival_Date' is currently an 'object' (string) type
# and must be converted to a proper datetime format for time-series analysis.
commodity_prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75207044 entries, 0 to 75207043
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   State           object 
 1   District        object 
 2   Market          object 
 3   Commodity       object 
 4   Variety         object 
 5   Grade           object 
 6   Arrival_Date    object 
 7   Min_Price       float64
 8   Max_Price       float64
 9   Modal_Price     float64
 10  Commodity_Code  int64  
dtypes: float64(3), int64(1), object(7)
memory usage: 6.2+ GB


In [78]:
# Convert the 'Arrival_Date' column from its string representation to a pandas datetime object.
# This is a critical transformation that enables all subsequent time-based operations like
# filtering by year, creating rolling windows, and calculating seasonality.
commodity_prices_df['Arrival_Date'] = pd.to_datetime(commodity_prices_df['Arrival_Date'])

# Re-run the .info() method to verify that the data type conversion was successful.
# The 'Arrival_Date' column's Dtype should now be correctly listed as 'datetime64[ns]'.
commodity_prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75207044 entries, 0 to 75207043
Data columns (total 11 columns):
 #   Column          Dtype         
---  ------          -----         
 0   State           object        
 1   District        object        
 2   Market          object        
 3   Commodity       object        
 4   Variety         object        
 5   Grade           object        
 6   Arrival_Date    datetime64[ns]
 7   Min_Price       float64       
 8   Max_Price       float64       
 9   Modal_Price     float64       
 10  Commodity_Code  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(6)
memory usage: 6.2+ GB


### 3. Handling Missing Values

This section addresses missing data points in the price columns. A targeted imputation strategy is used to fill these gaps accurately, preserving the integrity of the dataset for analysis.

In [79]:
# Conduct an initial assessment to count the number of null (missing) values in each column.
# The results show a small number of nulls in 'Min_Price' and 'Max_Price' that need to be addressed.
commodity_prices_df.isnull().sum()

,0
State,0
District,0
Market,0
Commodity,0
Variety,0
Grade,0
Arrival_Date,0
Min_Price,1
Max_Price,60
Modal_Price,0


#### 3.1 Imputing `Min_Price`

A single missing value was found in `Min_Price`. A highly specific mean, calculated from records with the same commodity and location, will be used for imputation. This is far more accurate than using a global average.

In [80]:
# Isolate and inspect the row containing the null 'Min_Price' to understand its context.
commodity_prices_df[commodity_prices_df['Min_Price'].isnull()]

,State,District,Market,Commodity,Variety,Grade,Arrival_Date,Min_Price,Max_Price,Modal_Price,Commodity_Code
124491,Chattisgarh,Gariyaband,Rajim,Paddy (Dhan)(Common),1001,FAQ,2007-01-20,NaN,607.0,603.0,2


In [81]:
# Create a filter to select only the data from the same market and for the same commodity as the row with the missing value.
min_price_imputation_filter = commodity_prices_df[
    (commodity_prices_df['State'] == 'Chattisgarh') &
    (commodity_prices_df['District'] == 'Gariyaband') &
    (commodity_prices_df['Market'] == 'Rajim') &
    (commodity_prices_df['Commodity'] == 'Paddy (Dhan)(Common)')
]

# Calculate the mean 'Min_Price' from this highly targeted subset of the data.
imputation_value = min_price_imputation_filter['Min_Price'].mean().round(2)

# Fill the single missing 'Min_Price' value. 'inplace=True' modifies the DataFrame directly.
commodity_prices_df['Min_Price'].fillna(imputation_value, inplace=True)

/tmp/ipython-input-334019736.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  commodity_prices_df['Min_Price'].fillna(imputation_value, inplace=True)


#### 3.2 Imputing `Max_Price`

For the 60 missing values in `Max_Price`, a more scalable approach is needed. The `groupby().transform()` method is used to fill each missing value with the mean of its specific group (based on location, commodity, and variety), automating the targeted imputation logic.

In [82]:
# Use the powerful `groupby().transform()` pattern to fill null 'Max_Price' values.
# For each row with a null value, this calculates the mean 'Max_Price' of all other rows with the same
# State, District, Market, Commodity, and Variety, and uses that specific mean to fill the gap.
commodity_prices_df['Max_Price'] = commodity_prices_df.groupby(
    ['State', 'District', 'Market', 'Commodity', 'Variety']
)['Max_Price'].transform(lambda x: x.fillna(x.mean()))

# As a robust fallback, if any group was entirely null (had no prices to calculate a mean),
# we fill any remaining nulls with the global mean for that commodity.
commodity_prices_df['Max_Price'].fillna(commodity_prices_df.groupby('Commodity')['Max_Price'].transform('mean'), inplace=True)

/tmp/ipython-input-2421332740.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  commodity_prices_df['Max_Price'].fillna(commodity_prices_df.groupby('Commodity')['Max_Price'].transform('mean'), inplace=True)


#### 3.3 Final Verification

In [83]:
# Perform a final check to confirm that all null values in the price columns have been successfully handled.
# The output should now show '0' for all price-related columns.
commodity_prices_df.isnull().sum()

,0
State,0
District,0
Market,0
Commodity,0
Variety,0
Grade,0
Arrival_Date,0
Min_Price,0
Max_Price,0
Modal_Price,0


### 4. Feature Engineering

With the data cleaned, we can now engineer new features (columns) that will be critical for our analysis. This involves categorizing commodities and extracting time-based information from the `Arrival_Date`.

#### 4.1 Categorizing Commodities

To enable analysis by category (e.g., Grains, Vegetables, Spices), we will classify each commodity based on keywords.

In [84]:
# Define a master dictionary of categorization keywords.
# The order is intentionally structured from most specific (e.g., 'Pulses') to more general
# to ensure the most accurate classification for each commodity.
CATEGORIES_KEYWORDS = {
    'Pulses': ['gram', 'dal', 'moong', 'arhar', 'urd', 'masur', 'lentil', 'peas', 'lobia', 'moath'],
    'Grains': ['wheat', 'rice', 'maize', 'jowar', 'bajra', 'barley', 'ragi', 'millet', 'paddy', 'soji'],
    'Spices': ['ginger', 'coriander', 'turmeric', 'pepper', 'jeera', 'cummin', 'chillies', 'chilly',
               'soanf', 'methi', 'ajwan', 'cinamon', 'cloves', 'cardamoms', 'bay leaf'],
    'Vegetables': ['onion', 'potato', 'brinjal', 'cauliflower', 'tomato', 'garlic', 'cabbage',
                   'capsicum', 'carrot', 'gourd', 'pumpkin', 'beans', 'cucumber', 'raddish',
                   'spinach', 'yam', 'beetroot', 'ladies finger', 'lemon', 'lime', 'knool khol', 'drumstick'],
    'Fruits': ['grapes', 'orange', 'pineapple', 'apple', 'banana', 'mango', 'guava', 'papaya',
               'pomegranate', 'mosambi', 'chikoos', 'sapota', 'watermelon', 'musk melon', 'litchi',
               'fig', 'plum', 'cherry', 'peach'],
    'Oilseeds': ['sunflower', 'mustard', 'soyabean', 'sesamum', 'groundnut', 'castor', 'linseed',
                 'niger', 'toria', 'sarson'],
    'Nuts': ['arecanut', 'cashewnuts', 'coconut', 'almond', 'walnut', 'copra'],
    'Fibers': ['cotton', 'jute', 'sunhemp'],
    'Sugar/Jaggery': ['sugar', 'gur', 'jaggery', 'sugarcane'],
    'Tea/Coffee': ['tea', 'coffee'],
    'Livestock': ['sheep', 'goat', 'cow', 'bull', 'ox', 'buffalo', 'egg', 'hen', 'cock', 'pigs', 'ram']
}

# Define a function to classify a commodity name based on the master dictionary.
def classify_commodity(commodity_name):
    """
    Iterates through the keyword dictionary and returns the first matching category.
    Returns 'Other' if no match is found.
    """
    name_lower = commodity_name.lower()
    for category, keywords in CATEGORIES_KEYWORDS.items():
        if any(keyword in name_lower for keyword in keywords):
            return category
    return 'Other'

# Apply this function to the 'Commodity' column to create the new 'Category' feature.
commodity_prices_df['Category'] = commodity_prices_df['Commodity'].apply(classify_commodity)

# As a sanity check, display the count of records for each new category to ensure the classification worked as expected.
print("--- Commodity Categorization Complete ---")
print(commodity_prices_df['Category'].value_counts())

--- Commodity Categorization Complete ---
Category
Vegetables       24928851
Grains           12289341
Pulses           10447648
Other             8930748
Fruits            6971858
Oilseeds          4958792
Spices            2956492
Nuts              1173931
Fibers            1063525
Sugar/Jaggery      941288
Livestock          517381
Tea/Coffee          27189
Name: count, dtype: int64


#### 4.2 Creating Time-Based Features

To facilitate time-series analysis, we extract the year, month number, and month name from the `Arrival_Date` column. These features are essential for grouping and aggregating data by specific time periods.

In [85]:
# Extract the year from the 'Arrival_Date' column.
commodity_prices_df['Year'] = commodity_prices_df['Arrival_Date'].dt.year

# Extract the month number (1-12) from the 'Arrival_Date' column.
commodity_prices_df['Month'] = commodity_prices_df['Arrival_Date'].dt.month

# Extract the full month name (e.g., 'January') from the 'Arrival_Date' column.
commodity_prices_df['Month_Name'] = commodity_prices_df['Arrival_Date'].dt.month_name()

# Display the first few rows with the new time-based columns to verify their creation.
print("\n--- Time-Based Feature Creation Complete ---")
commodity_prices_df[['Arrival_Date', 'Year', 'Month', 'Month_Name', 'Category']].head()


--- Time-Based Feature Creation Complete ---


,Arrival_Date,Year,Month,Month_Name,Category
0,2007-01-01,2007,1,January,Other
1,2007-01-01,2007,1,January,Oilseeds
2,2007-01-01,2007,1,January,Pulses
3,2007-01-01,2007,1,January,Grains
4,2007-01-01,2007,1,January,Grains


### 5. Saving the Final, Cleaned Dataset

The data cleaning and feature engineering process is now complete. The final, enriched DataFrame is saved to the Parquet format. Parquet is chosen over CSV because it is significantly faster, creates smaller file sizes, and preserves data types (like `datetime`), making it ideal for analytical workflows.

In [86]:
# Save the fully cleaned and enriched DataFrame to a Parquet file.
# This file will serve as the starting point for all subsequent analysis,
# eliminating the need to repeat the cleaning steps.
commodity_prices_df.to_parquet('cleaned_commodity_prices.parquet', index=False)

print("\nSuccessfully saved the final DataFrame to 'cleaned_commodity_prices.parquet'")


Successfully saved the final DataFrame to 'cleaned_commodity_prices.parquet'
